In [2]:
from pathlib import Path
import pandas as pd

# ================= CONFIGURAÇÃO =================
PASTA_EXCEL = Path(r"C:\projetos\superveniencia_piscofins\data\Output\Resumos\excel")
PASTA_PDF   = Path(r"C:\projetos\superveniencia_piscofins\data\Output\Resumos\pdf")
ARQUIVO_TXT = Path(r"C:\projetos\superveniencia_piscofins\data\Input\Resumos\contratos.txt")
SAIDA_EXCEL = Path(r"C:\projetos\superveniencia_piscofins\data\Output\Resumos\relatorio_faltantes.xlsx")
# ===============================================

def ler_contratos_txt(caminho_txt: Path) -> list[str]:
    with caminho_txt.open("r", encoding="utf-8") as f:
        return [linha.strip() for linha in f if linha.strip()]

def contratos_em_pasta(pasta: Path, extensoes: tuple[str]) -> set[str]:
    contratos = set()
    for arquivo in pasta.iterdir():
        if arquivo.is_file() and arquivo.suffix.lower() in extensoes:
            nome = arquivo.stem
            contratos.add(nome)
    return contratos

def contrato_existe_no_nome(contrato: str, nomes_arquivos: set[str]) -> bool:
    return any(contrato in nome for nome in nomes_arquivos)

def main():
    contratos_txt = ler_contratos_txt(ARQUIVO_TXT)

    arquivos_excel = contratos_em_pasta(PASTA_EXCEL, (".xlsx", ".xls"))
    arquivos_pdf   = contratos_em_pasta(PASTA_PDF, (".pdf",))

    registros = []

    for contrato in contratos_txt:
        existe_excel = contrato_existe_no_nome(contrato, arquivos_excel)
        existe_pdf   = contrato_existe_no_nome(contrato, arquivos_pdf)

        if not existe_excel and not existe_pdf:
            status = "Faltando Excel e PDF"
        elif not existe_excel:
            status = "Faltando Excel"
        elif not existe_pdf:
            status = "Faltando PDF"
        else:
            status = "OK"

        registros.append({
            "Contrato": contrato,
            "Existe na pasta Excel": "Sim" if existe_excel else "Não",
            "Existe na pasta PDF": "Sim" if existe_pdf else "Não",
            "Status": status
        })

    df = pd.DataFrame(registros)

    # Mantém apenas os que têm algum problema
    df_faltantes = df[df["Status"] != "OK"]

    df_faltantes.to_excel(SAIDA_EXCEL, index=False)

    print(f"Relatório gerado em: {SAIDA_EXCEL}")

if __name__ == "__main__":
    main()


Relatório gerado em: C:\projetos\superveniencia_piscofins\data\Output\Resumos\relatorio_faltantes.xlsx


0 A ESQUERDA

In [1]:
from pathlib import Path
import pandas as pd
import re
from collections import defaultdict

# ================= CONFIGURAÇÃO =================
PASTA_EXCEL = Path(r"C:\projetos\superveniencia_piscofins\data\Output\Resumos\excel")
ARQUIVO_TXT = Path(r"C:\projetos\superveniencia_piscofins\data\Input\Resumos\contratos.txt")
SAIDA_EXCEL = Path(r"C:\projetos\superveniencia_piscofins\data\Output\Resumos\relatorio_faltantes_excel.xlsx")
# ===============================================

RE_NUM = re.compile(r"\d+")

def normalizar_7dig(s: str) -> str | None:
    dig = re.sub(r"\D", "", str(s).strip())
    if not dig:
        return None
    return dig.zfill(7)

def ler_contratos_txt(caminho_txt: Path) -> list[str]:
    contratos = []
    with caminho_txt.open("r", encoding="utf-8", errors="replace") as f:
        for linha in f:
            linha = linha.strip()
            if not linha:
                continue
            c = normalizar_7dig(linha)
            if c:
                contratos.append(c)

    # remove duplicados preservando ordem
    vistos = set()
    out = []
    for c in contratos:
        if c not in vistos:
            vistos.add(c)
            out.append(c)
    return out

def mapear_contratos_na_pasta_excel(pasta: Path, extensoes: tuple[str, ...]) -> dict[str, set[str]]:
    """
    Retorna: contrato (7 dígitos) -> {arquivos Excel que contêm esse contrato no nome}
    """
    mapa = defaultdict(set)
    for arquivo in pasta.iterdir():
        if not arquivo.is_file():
            continue
        if arquivo.suffix.lower() not in extensoes:
            continue

        for n in RE_NUM.findall(arquivo.stem):
            c = normalizar_7dig(n)
            if c:
                mapa[c].add(arquivo.name)

    return mapa

def main():
    contratos_txt = ler_contratos_txt(ARQUIVO_TXT)
    mapa_excel = mapear_contratos_na_pasta_excel(PASTA_EXCEL, (".xlsx", ".xls"))

    registros = []
    for contrato in contratos_txt:
        existe_excel = contrato in mapa_excel

        status = "OK" if existe_excel else "Faltando Excel"

        registros.append({
            "Contrato": contrato,
            "Existe na pasta Excel": "Sim" if existe_excel else "Não",
            "Status": status,
            "Arquivos Excel (exemplos)": " | ".join(sorted(list(mapa_excel.get(contrato, [])))[:5]),
        })

    df = pd.DataFrame(registros)

    # Mantém apenas os que têm algum problema
    df_faltantes = df[df["Status"] != "OK"].copy()

    SAIDA_EXCEL.parent.mkdir(parents=True, exist_ok=True)
    df_faltantes.to_excel(SAIDA_EXCEL, index=False)

    print(f"Relatório gerado em: {SAIDA_EXCEL}")
    print(f"Total contratos TXT: {len(contratos_txt)}")
    print(f"Total faltando Excel: {len(df_faltantes)}")

if __name__ == "__main__":
    main()


Relatório gerado em: C:\projetos\superveniencia_piscofins\data\Output\Resumos\relatorio_faltantes_excel.xlsx
Total contratos TXT: 22881
Total faltando Excel: 1


In [11]:
from src.anexo_c.process_dashboard import process_dashboard
from pathlib import Path
import pandas as pd
DASHBOARD_CSV = Path(r"data/input/Diligencia_Contratos.csv")


In [18]:
TEMPLATE_PATH = Path(r"docs/template_csll.xlsx")
OUTPUT_DIR = Path(r"data/output2")
INPUT_DIR = Path(r"data/input")

TAX_FILTERS = {
    "IRPJ": ["RAIR", "Exclusão", "Adição"],
    "CS": ["RAIR", "Exclusão", "Adição"],
}
cls = process_dashboard(TAX_FILTERS, str(TEMPLATE_PATH))


In [12]:
dashboard = pd.read_csv(
    DASHBOARD_CSV,
    sep=";",
    encoding="latin1",
    dtype={"NumContrato": "int64"},
)

In [16]:
dash_filter = dashboard[dashboard["NumContrato"] == 386721]

In [20]:
teste = cls.replicate_years(dash_filter, tax_cols=["IRPJ", "CS"])

In [21]:
teste['Ano'].unique()

array([2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024])

In [28]:
df = dash_filter.copy()
tax_cols = ["IRPJ", "CS"]
fill_value = 0

In [29]:
AGRUPAMENTO_COLS = ["Ano", "Conta_Nome", "Cosif_Nome"]
AGRUPAMENTO_VALUES = ["ValorDebito", "ValorCredito", "Movimentacao"]
DESC_COL = "Descrição"
ANOMES_COL = "AnoMes"
COL_VALOR = "Movimentacao"
ANO_COL = "Ano"

VALUE_COLS = ["ValorDebito", "ValorCredito", "Movimentacao"]
GROUP_COLS = ["Conta_Nome", "Cosif_Nome"]

df = df.copy()
df.loc[:, ANO_COL] = df[ANOMES_COL] // 100

# resolve tax_cols
if tax_cols is None:
    tax_cols = getattr("tax_cols", None) or getattr("tax_col", None)
    if tax_cols is None:
        raise ValueError(
            "Informe tax_cols (ex: ['IRPJ','CS']) ou defina tax_cols/tax_col."
        )
if isinstance(tax_cols, str):
    tax_cols = [tax_cols]

key_cols = tax_cols + GROUP_COLS

# agrega original (inclui 9999)
df_agg = df.groupby(key_cols + [ANO_COL], as_index=False)[VALUE_COLS].sum()

# separa 9999 (fica no output, mas não entra na replicação)
df_9999 = df_agg[df_agg[ANO_COL] == 9999].copy()
df_real = df_agg[df_agg[ANO_COL] != 9999].copy()

# se não tem anos reais, não tem o que "replicar"; devolve só o 9999
if df_real.empty:
    print(df_9999.reset_index(drop=True))

# intervalo de anos reais (sem 9999)
min_year = int(df_real[ANO_COL].min())
max_year = int(df_real[ANO_COL].max())
years = list(range(min_year, max_year + 1))

# combos baseadas nos grupos que existem nos anos reais
# (assim a gente não cria “linhas novas” só porque existe 9999)
groups = df_real[key_cols].drop_duplicates().reset_index(drop=True)

years_df = pd.DataFrame({ANO_COL: years})

# cross join grupos × anos reais
groups["key"] = 1
years_df["key"] = 1
full = groups.merge(years_df, on="key").drop("key", axis=1)

# expande anos faltantes
df_full_real = full.merge(df_real, on=key_cols + [ANO_COL], how="left")
df_full_real[VALUE_COLS] = df_full_real[VALUE_COLS].fillna(fill_value)

# junta de volta o 9999 (sem replicar)
out = pd.concat([df_full_real, df_9999], ignore_index=True)

print(out)

         IRPJ        CS  ... ValorCredito Movimentacao
0    Exclusão  Exclusão  ...         0.00         0.00
1    Exclusão  Exclusão  ...         0.00         0.00
2    Exclusão  Exclusão  ...     10799.31      -169.19
3    Exclusão  Exclusão  ...      4997.17         0.00
4    Exclusão  Exclusão  ...      4658.47         0.00
..        ...       ...  ...          ...          ...
373      RAIR      RAIR  ...         0.00         0.00
374      RAIR      RAIR  ...         0.00         0.00
375      RAIR      RAIR  ...         0.00         0.00
376      RAIR      RAIR  ...         0.00         0.00
377      RAIR      RAIR  ...         0.00         0.00

[378 rows x 8 columns]


In [30]:
df_9999

,IRPJ,CS,Conta_Nome,Cosif_Nome,Ano,ValorDebito,ValorCredito,Movimentacao
